In [1]:
# ============================================================================
# 1. IMPORT REQUIRED LIBRARIES
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [2]:
# ============================================================================
# 2. LOAD AND EXPLORE THE DATASET
# ============================================================================

print("="*80)
print("STEP 1: LOADING AND EXPLORING THE DATASET")
print("="*80)

# Load the dataset (replace with your actual file path)
# For demonstration, we'll create a sample dataset similar to loan prediction data
# In practice, you would use: df = pd.read_csv('loan_dataset.csv')

def create_sample_dataset(n_samples=1000):
    """Create a sample dataset for demonstration purposes"""
    np.random.seed(42)

    data = {
        'ApplicantIncome': np.random.normal(5000, 2000, n_samples),
        'CoapplicantIncome': np.random.normal(2000, 1000, n_samples),
        'LoanAmount': np.random.normal(150, 50, n_samples),
        'Loan_Amount_Term': np.random.choice([360, 180, 120, 240], n_samples),
        'Credit_History': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),
        'Property_Area': np.random.choice(['Urban', 'Semiurban', 'Rural'], n_samples),
        'Gender': np.random.choice(['Male', 'Female'], n_samples),
        'Married': np.random.choice(['Yes', 'No'], n_samples),
        'Education': np.random.choice(['Graduate', 'Not Graduate'], n_samples),
        'Self_Employed': np.random.choice(['Yes', 'No'], n_samples, p=[0.1, 0.9])
    }

    # Create relationships to make the dataset more realistic
    df = pd.DataFrame(data)

    # Add some relationships between features and target
    base_amount = (df['ApplicantIncome'] * 0.02 +
                   df['CoapplicantIncome'] * 0.01 +
                   df['Credit_History'] * 50 +
                   np.random.normal(0, 20, n_samples))

    # Adjust for property area
    area_multiplier = {'Urban': 1.2, 'Semiurban': 1.0, 'Rural': 0.8}
    df['LoanAmount'] = base_amount * df['Property_Area'].map(area_multiplier)

    # Ensure LoanAmount is positive
    df['LoanAmount'] = np.abs(df['LoanAmount'])

    return df

# Create or load the dataset
print("Creating sample dataset...")
df = create_sample_dataset(1000)

# Display basic information about the dataset
print("\nDataset Shape:", df.shape)
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nBasic Statistics:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

STEP 1: LOADING AND EXPLORING THE DATASET
Creating sample dataset...

Dataset Shape: (1000, 10)

First 5 rows of the dataset:
   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0      5993.428306        3399.355437  210.796073               180   
1      4723.471398        2924.633683  192.016134               240   
2      6295.377076        2059.630370  209.538067               180   
3      8046.059713        1353.063222  207.287599               180   
4      4531.693251        2698.223314  140.125420               360   

   Credit_History Property_Area  Gender Married     Education Self_Employed  
0               1     Semiurban    Male     Yes  Not Graduate            No  
1               1         Urban  Female      No      Graduate            No  
2               1     Semiurban  Female     Yes  Not Graduate            No  
3               0         Urban  Female      No  Not Graduate            No  
4               1         Rural    Male      No      Grad

In [3]:
# ============================================================================
# 3. DATA PREPROCESSING
# ============================================================================

print("\n" + "="*80)
print("STEP 2: DATA PREPROCESSING")
print("="*80)

# Handle missing values (if any)
print("\nHandling missing values...")
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)

print("Missing values after handling:")
print(df.isnull().sum())

# Encode categorical variables
print("\nEncoding categorical variables...")
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {categorical_cols}")

# Use one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
print(f"\nShape after encoding: {df_encoded.shape}")

# Separate features and target
target_column = 'LoanAmount'
X = df_encoded.drop(target_column, axis=1)
y = df_encoded[target_column]

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Standardize numerical features
print("\nStandardizing numerical features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

print("Feature scaling completed!")


STEP 2: DATA PREPROCESSING

Handling missing values...
Missing values after handling:
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Gender               0
Married              0
Education            0
Self_Employed        0
dtype: int64

Encoding categorical variables...
Categorical columns: ['Property_Area', 'Gender', 'Married', 'Education', 'Self_Employed']

Shape after encoding: (1000, 11)

Features shape: (1000, 10)
Target shape: (1000,)

Training set size: (800, 10)
Test set size: (200, 10)

Standardizing numerical features...
Feature scaling completed!



STEP 3: EXPLORATORY DATA ANALYSIS


EDA visualizations saved as 'eda_visualizations.png'


In [4]:
# ============================================================================
# 4. EXPLORATORY DATA ANALYSIS (EDA) VISUALIZATIONS
# ============================================================================

print("\n" + "="*80)
print("STEP 3: EXPLORATORY DATA ANALYSIS")
print("="*80)

# Create a figure with multiple subplots for EDA
fig = plt.figure(figsize=(20, 12))

# 1. Target Variable Distribution
plt.subplot(2, 3, 1)
sns.histplot(y, kde=True, bins=30)
plt.title('Distribution of Loan Amount (Target Variable)', fontsize=14, fontweight='bold')
plt.xlabel('Loan Amount')
plt.ylabel('Frequency')

# 2. Box plot of target variable
plt.subplot(2, 3, 2)
sns.boxplot(y=y)
plt.title('Box Plot of Loan Amount', fontsize=14, fontweight='bold')
plt.xlabel('Loan Amount')

# 3. Feature vs Target scatter plots (select top numerical features)
numerical_features = X.select_dtypes(include=[np.number]).columns[:3]
plt.subplot(2, 3, 3)
for i, feature in enumerate(numerical_features):
    plt.scatter(X[feature][:100], y[:100], alpha=0.6, label=feature, s=30)
plt.title('Feature vs Target Scatter Plots', fontsize=14, fontweight='bold')
plt.xlabel('Feature Value')
plt.ylabel('Loan Amount')
plt.legend()

# 4. Correlation heatmap
plt.subplot(2, 3, 4)
# Combine features and target for correlation
corr_data = pd.concat([X[numerical_features], y], axis=1)
corr_matrix = corr_data.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Heatmap', fontsize=14, fontweight='bold')

# 5. Feature distributions
plt.subplot(2, 3, 5)
for feature in numerical_features[:2]:
    sns.kdeplot(data=X, x=feature, label=feature)
plt.title('Feature Distributions', fontsize=14, fontweight='bold')
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

# 6. Summary statistics table
plt.subplot(2, 3, 6)
plt.axis('tight')
plt.axis('off')
summary_stats = y.describe().round(2)
cell_text = [[f"{stat}: {value}"] for stat, value in summary_stats.items()]
plt.table(cellText=cell_text, colLabels=['Target Variable Statistics'],
          loc='center', cellLoc='left')
plt.title('Summary Statistics', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('eda_visualizations.png', dpi=300, bbox_inches='tight')
plt.show()
print("EDA visualizations saved as 'eda_visualizations.png'")

In [5]:
# ============================================================================
# 5. TRAIN BASELINE LINEAR REGRESSION
# ============================================================================

print("\n" + "="*80)
print("STEP 4: TRAINING BASELINE LINEAR REGRESSION")
print("="*80)

# Train Linear Regression
lr_model = LinearRegression()
start_time = time.time()
lr_model.fit(X_train_scaled, y_train)
lr_training_time = time.time() - start_time

# Make predictions
lr_train_pred = lr_model.predict(X_train_scaled)
lr_test_pred = lr_model.predict(X_test_scaled)

# Calculate metrics for Linear Regression
def calculate_metrics(y_true, y_pred, model_name, dataset_type):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {'Model': model_name, 'Dataset': dataset_type,
            'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2}

lr_train_metrics = calculate_metrics(y_train, lr_train_pred, 'Linear Regression', 'Training')
lr_test_metrics = calculate_metrics(y_test, lr_test_pred, 'Linear Regression', 'Test')

print("\nLinear Regression Performance:")
print(f"Training - MAE: {lr_train_metrics['MAE']:.4f}, MSE: {lr_train_metrics['MSE']:.4f}, "
      f"RMSE: {lr_train_metrics['RMSE']:.4f}, R²: {lr_train_metrics['R2']:.4f}")
print(f"Test - MAE: {lr_test_metrics['MAE']:.4f}, MSE: {lr_test_metrics['MSE']:.4f}, "
      f"RMSE: {lr_test_metrics['RMSE']:.4f}, R²: {lr_test_metrics['R2']:.4f}")
print(f"Training Time: {lr_training_time:.4f} seconds")


STEP 4: TRAINING BASELINE LINEAR REGRESSION

Linear Regression Performance:
Training - MAE: 16.6967, MSE: 455.3593, RMSE: 21.3392, R²: 0.8656
Test - MAE: 19.7953, MSE: 591.2153, RMSE: 24.3149, R²: 0.8156
Training Time: 0.0192 seconds


In [6]:
# ============================================================================
# 6. HYPERPARAMETER TUNING WITH CROSS-VALIDATION
# ============================================================================

print("\n" + "="*80)
print("STEP 5: HYPERPARAMETER TUNING WITH 5-FOLD CROSS-VALIDATION")
print("="*80)

# Define parameter grids as per the experiment PDF
param_grid_ridge = {'alpha': [0.01, 0.1, 1, 10, 100]}
param_grid_lasso = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
param_grid_enet = {'alpha': [0.01, 0.1, 1, 10], 'l1_ratio': [0.2, 0.5, 0.8]}

# Create 5-fold cross-validation object
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store best models and results
best_models = {}
cv_results = {}

# Ridge Regression Tuning
print("\n" + "-"*40)
print("Tuning Ridge Regression...")
print("-"*40)
ridge_grid = GridSearchCV(Ridge(), param_grid_ridge, cv=cv, scoring='r2', verbose=1, n_jobs=-1)
ridge_grid.fit(X_train_scaled, y_train)
best_models['Ridge'] = ridge_grid.best_estimator_
cv_results['Ridge'] = {
    'best_params': ridge_grid.best_params_,
    'best_cv_r2': ridge_grid.best_score_,
    'cv_results': ridge_grid.cv_results_
}
print(f"Best parameters: {ridge_grid.best_params_}")
print(f"Best CV R²: {ridge_grid.best_score_:.4f}")

# Lasso Regression Tuning
print("\n" + "-"*40)
print("Tuning Lasso Regression...")
print("-"*40)
lasso_grid = GridSearchCV(Lasso(max_iter=10000), param_grid_lasso, cv=cv, scoring='r2', verbose=1, n_jobs=-1)
lasso_grid.fit(X_train_scaled, y_train)
best_models['Lasso'] = lasso_grid.best_estimator_
cv_results['Lasso'] = {
    'best_params': lasso_grid.best_params_,
    'best_cv_r2': lasso_grid.best_score_,
    'cv_results': lasso_grid.cv_results_
}
print(f"Best parameters: {lasso_grid.best_params_}")
print(f"Best CV R²: {lasso_grid.best_score_:.4f}")

# Elastic Net Tuning
print("\n" + "-"*40)
print("Tuning Elastic Net Regression...")
print("-"*40)
enet_grid = GridSearchCV(ElasticNet(max_iter=10000), param_grid_enet, cv=cv, scoring='r2', verbose=1, n_jobs=-1)
enet_grid.fit(X_train_scaled, y_train)
best_models['Elastic Net'] = enet_grid.best_estimator_
cv_results['Elastic Net'] = {
    'best_params': enet_grid.best_params_,
    'best_cv_r2': enet_grid.best_score_,
    'cv_results': enet_grid.cv_results_
}
print(f"Best parameters: {enet_grid.best_params_}")
print(f"Best CV R²: {enet_grid.best_score_:.4f}")


STEP 5: HYPERPARAMETER TUNING WITH 5-FOLD CROSS-VALIDATION

----------------------------------------
Tuning Ridge Regression...
----------------------------------------
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best parameters: {'alpha': 1}
Best CV R²: 0.8609

----------------------------------------
Tuning Lasso Regression...
----------------------------------------
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best parameters: {'alpha': 0.1}
Best CV R²: 0.8609

----------------------------------------
Tuning Elastic Net Regression...
----------------------------------------
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters: {'alpha': 0.01, 'l1_ratio': 0.8}
Best CV R²: 0.8609


In [7]:
# ============================================================================
# 7. CREATE HYPERPARAMETER TUNING SUMMARY TABLE (Table 1)
# ============================================================================

print("\n" + "="*80)
print("TABLE 1: HYPERPARAMETER TUNING SUMMARY")
print("="*80)

table1_data = []
for model_name, results in cv_results.items():
    table1_data.append({
        'Model': model_name,
        'Search Method': 'Grid Search',
        'Best Parameters': str(results['best_params']),
        'Best CV R²': f"{results['best_cv_r2']:.4f}"
    })

table1_df = pd.DataFrame(table1_data)
print("\n", table1_df.to_string(index=False))


TABLE 1: HYPERPARAMETER TUNING SUMMARY

       Model Search Method                  Best Parameters Best CV R²
      Ridge   Grid Search                     {'alpha': 1}     0.8609
      Lasso   Grid Search                   {'alpha': 0.1}     0.8609
Elastic Net   Grid Search {'alpha': 0.01, 'l1_ratio': 0.8}     0.8609


In [8]:
# ============================================================================
# 8. EVALUATE ALL MODELS WITH CROSS-VALIDATION (Table 2)
# ============================================================================

print("\n" + "="*80)
print("TABLE 2: CROSS-VALIDATION PERFORMANCE (K=5)")
print("="*80)

def get_cv_scores(model, X, y, cv):
    """Get detailed CV scores for a model"""
    from sklearn.model_selection import cross_val_score

    mae_scores = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    mse_scores = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(mse_scores)
    r2_scores = cross_val_score(model, X, y, cv=cv, scoring='r2')

    return {
        'MAE': mae_scores.mean(),
        'MSE': mse_scores.mean(),
        'RMSE': rmse_scores.mean(),
        'R2': r2_scores.mean()
    }

# Get CV scores for all models
cv_scores = {}
models_for_cv = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': best_models['Ridge'],
    'Lasso Regression': best_models['Lasso'],
    'Elastic Net Regression': best_models['Elastic Net']
}

for model_name, model in models_for_cv.items():
    cv_scores[model_name] = get_cv_scores(model, X_train_scaled, y_train, cv)

# Create Table 2
table2_data = []
for model_name, scores in cv_scores.items():
    table2_data.append({
        'Model': model_name,
        'MAE': f"{scores['MAE']:.4f}",
        'MSE': f"{scores['MSE']:.4f}",
        'RMSE': f"{scores['RMSE']:.4f}",
        'R²': f"{scores['R2']:.4f}"
    })

table2_df = pd.DataFrame(table2_data)
print("\n", table2_df.to_string(index=False))


TABLE 2: CROSS-VALIDATION PERFORMANCE (K=5)

                  Model     MAE      MSE    RMSE     R²
     Linear Regression 16.9225 466.1553 21.5733 0.8608
      Ridge Regression 16.9185 466.1173 21.5723 0.8609
      Lasso Regression 16.9088 465.9521 21.5676 0.8609
Elastic Net Regression 16.9164 466.0894 21.5715 0.8609


In [11]:
# ============================================================================
# 9. TEST SET PERFORMANCE COMPARISON (Table 3)
# ============================================================================

print("\n" + "="*80)
print("TABLE 3: TEST SET PERFORMANCE")
print("="*80)

# Make sure Linear Regression is fitted
# If you haven't fitted it earlier, fit it now
if not hasattr(lr_model, 'coef_'):
    print("Fitting Linear Regression model...")
    lr_model.fit(X_train, y_train)

# Create models_for_cv with ALL models fitted
models_for_cv = {
    'Linear Regression': lr_model,  # Now this is fitted
    'Ridge Regression': best_models['Ridge'],  # Already fitted from GridSearch
    'Lasso Regression': best_models['Lasso'],  # Already fitted from GridSearch
    'Elastic Net Regression': best_models['Elastic Net']  # Already fitted from GridSearch
}

# Make predictions with all models
test_predictions = {}
train_predictions = {}

for model_name, model in models_for_cv.items():
    print(f"Making predictions for {model_name}...")
    # Training predictions
    train_predictions[model_name] = model.predict(X_train)
    # Test predictions
    test_predictions[model_name] = model.predict(X_test)

# Calculate test metrics
table3_data = []
for model_name, y_pred in test_predictions.items():
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    table3_data.append({
        'Model': model_name,
        'MAE': f"{mae:.4f}",
        'MSE': f"{mse:.4f}",
        'RMSE': f"{rmse:.4f}",
        'R²': f"{r2:.4f}"
    })

table3_df = pd.DataFrame(table3_data)
print("\n", table3_df.to_string(index=False))


TABLE 3: TEST SET PERFORMANCE
Making predictions for Linear Regression...
Making predictions for Ridge Regression...
Making predictions for Lasso Regression...
Making predictions for Elastic Net Regression...

                  Model         MAE              MSE        RMSE             R²
     Linear Regression 217142.5007 53148996574.2541 230540.6614 -16577344.4450
      Ridge Regression 216878.9948 53020138032.9994 230261.0215 -16537153.0455
      Lasso Regression 216518.7246 52852675729.7880 229897.0981 -16484921.0068
Elastic Net Regression 216671.6234 52919517505.8478 230042.4254 -16505769.1748


In [12]:
# ============================================================================
# 10. COEFFICIENT COMPARISON (Table 4)
# ============================================================================

print("\n" + "="*80)
print("TABLE 4: COEFFICIENT COMPARISON")
print("="*80)

# Get top 5 most important features for comparison
feature_importance = np.abs(lr_model.coef_)
top_feature_indices = np.argsort(feature_importance)[-5:][::-1]
top_features = X.columns[top_feature_indices]

# Collect coefficients for all models
coefficients_data = []
for idx, feature in enumerate(top_features):
    row = {'Feature': feature}

    # Linear Regression coefficients
    row['Linear'] = f"{lr_model.coef_[top_feature_indices[idx]]:.4f}"

    # Ridge coefficients
    row['Ridge'] = f"{best_models['Ridge'].coef_[top_feature_indices[idx]]:.4f}"

    # Lasso coefficients
    row['Lasso'] = f"{best_models['Lasso'].coef_[top_feature_indices[idx]]:.4f}"

    # Elastic Net coefficients
    row['Elastic Net'] = f"{best_models['Elastic Net'].coef_[top_feature_indices[idx]]:.4f}"

    coefficients_data.append(row)

table4_df = pd.DataFrame(coefficients_data)
print("\n", table4_df.to_string(index=False))


TABLE 4: COEFFICIENT COMPARISON

                 Feature  Linear   Ridge   Lasso Elastic Net
        ApplicantIncome 39.1592 39.1119 39.0822     39.0775
    Property_Area_Urban 28.4846 28.4252 28.2720     28.3726
         Credit_History 24.2053 24.1769 24.1165     24.1528
Property_Area_Semiurban 14.7393 14.6934 14.5271     14.6488
      CoapplicantIncome 10.7675 10.7539 10.6608     10.7373



STEP 6: GENERATING REQUIRED VISUALIZATIONS


All visualizations saved as 'regression_analysis_visualizations.png'


In [13]:
# ============================================================================
# 11. REQUIRED VISUALIZATIONS
# ============================================================================

print("\n" + "="*80)
print("STEP 6: GENERATING REQUIRED VISUALIZATIONS")
print("="*80)

# Create a figure with all required plots
fig = plt.figure(figsize=(20, 15))

# 1. Predicted vs Actual Values Plot
plt.subplot(2, 3, 1)
for model_name, y_pred in test_predictions.items():
    plt.scatter(y_test, y_pred, alpha=0.5, s=20, label=model_name)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Loan Amount', fontsize=12)
plt.ylabel('Predicted Loan Amount', fontsize=12)
plt.title('Predicted vs Actual Values', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

# 2. Residual Plot (using Lasso as an example)
plt.subplot(2, 3, 2)
residuals = y_test - test_predictions['Lasso Regression']
plt.scatter(test_predictions['Lasso Regression'], residuals, alpha=0.6, s=30)
plt.hlines(y=0, xmin=test_predictions['Lasso Regression'].min(),
           xmax=test_predictions['Lasso Regression'].max(), colors='r', linestyles='--', lw=2)
plt.xlabel('Predicted Loan Amount', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.title('Residual Plot (Lasso Regression)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# 3. Training Error vs Validation Error Plot
plt.subplot(2, 3, 3)
alphas = [0.001, 0.01, 0.1, 1, 10]
train_errors = []
val_errors = []

for alpha in alphas:
    lasso_temp = Lasso(alpha=alpha, max_iter=10000)
    lasso_temp.fit(X_train_scaled, y_train)

    train_pred = lasso_temp.predict(X_train_scaled)
    val_pred = lasso_temp.predict(X_test_scaled)

    train_errors.append(mean_squared_error(y_train, train_pred))
    val_errors.append(mean_squared_error(y_test, val_pred))

plt.plot(alphas, train_errors, 'b-', marker='o', label='Training Error', linewidth=2)
plt.plot(alphas, val_errors, 'r-', marker='s', label='Validation Error', linewidth=2)
plt.xscale('log')
plt.xlabel('Alpha (Regularization Strength)', fontsize=12)
plt.ylabel('Mean Squared Error', fontsize=12)
plt.title('Training vs Validation Error', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Coefficient Comparison Bar Plot
plt.subplot(2, 3, 4)
x = np.arange(len(top_features))
width = 0.2

# Get coefficients for the bar plot
coef_lr = lr_model.coef_[top_feature_indices]
coef_ridge = best_models['Ridge'].coef_[top_feature_indices]
coef_lasso = best_models['Lasso'].coef_[top_feature_indices]
coef_enet = best_models['Elastic Net'].coef_[top_feature_indices]

plt.bar(x - 1.5*width, coef_lr, width, label='Linear', alpha=0.8)
plt.bar(x - 0.5*width, coef_ridge, width, label='Ridge', alpha=0.8)
plt.bar(x + 0.5*width, coef_lasso, width, label='Lasso', alpha=0.8)
plt.bar(x + 1.5*width, coef_enet, width, label='Elastic Net', alpha=0.8)

plt.xlabel('Features', fontsize=12)
plt.ylabel('Coefficient Value', fontsize=12)
plt.title('Coefficient Comparison Across Models', fontsize=14, fontweight='bold')
plt.xticks(x, [f'F{i+1}' for i in range(len(top_features))], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 5. R² Score Comparison
plt.subplot(2, 3, 5)
models = list(test_predictions.keys())
r2_scores = [r2_score(y_test, test_predictions[model]) for model in models]
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold']
bars = plt.bar(models, r2_scores, color=colors, alpha=0.8)
plt.ylabel('R² Score', fontsize=12)
plt.title('R² Score Comparison', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
for bar, score in zip(bars, r2_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# 6. Training Time Comparison
plt.subplot(2, 3, 6)
training_times = [lr_training_time,
                  ridge_grid.refit_time_,
                  lasso_grid.refit_time_,
                  enet_grid.refit_time_]
bars = plt.bar(models, training_times, color=colors, alpha=0.8)
plt.ylabel('Training Time (seconds)', fontsize=12)
plt.title('Model Training Time Comparison', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
for bar, time_val in zip(bars, training_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{time_val:.3f}s', ha='center', va='bottom', fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('regression_analysis_visualizations.png', dpi=300, bbox_inches='tight')
plt.show()
print("All visualizations saved as 'regression_analysis_visualizations.png'")

In [14]:
# ============================================================================
# 12. OVERFITTING AND UNDERFITTING ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("OVERFITTING AND UNDERFITTING ANALYSIS")
print("="*80)

print("\nAnalyzing the difference between training and validation errors:")
print("-" * 60)

for model_name in models_for_cv.keys():
    train_r2 = r2_score(y_train, train_predictions[model_name])
    test_r2 = r2_score(y_test, test_predictions[model_name])
    r2_diff = train_r2 - test_r2

    print(f"\n{model_name}:")
    print(f"  Training R²: {train_r2:.4f}")
    print(f"  Test R²: {test_r2:.4f}")
    print(f"  Difference: {r2_diff:.4f}")

    if r2_diff > 0.1:
        print(f"  → Indicates potential OVERFITTING (large gap between train and test performance)")
    elif r2_diff < -0.05:
        print(f"  → Indicates potential UNDERFITTING (test performance better than training)")
    else:
        print(f"  → Good generalization (small gap between train and test performance)")


OVERFITTING AND UNDERFITTING ANALYSIS

Analyzing the difference between training and validation errors:
------------------------------------------------------------

Linear Regression:
  Training R²: -16074096.4403
  Test R²: -16577344.4450
  Difference: 503248.0046
  → Indicates potential OVERFITTING (large gap between train and test performance)

Ridge Regression:
  Training R²: -16035121.4033
  Test R²: -16537153.0455
  Difference: 502031.6423
  → Indicates potential OVERFITTING (large gap between train and test performance)

Lasso Regression:
  Training R²: -15984134.3720
  Test R²: -16484921.0068
  Difference: 500786.6348
  → Indicates potential OVERFITTING (large gap between train and test performance)

Elastic Net Regression:
  Training R²: -16004660.4110
  Test R²: -16505769.1748
  Difference: 501108.7638
  → Indicates potential OVERFITTING (large gap between train and test performance)


In [17]:
# ============================================================================
# 13. BIAS-VARIANCE ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("BIAS-VARIANCE ANALYSIS")
print("="*80)

print("""
Bias-Variance Trade-off Analysis:

1. Linear Regression:
   - Bias: Low (flexible model that fits training data well)
   - Variance: High (small changes in data can cause large changes in predictions)
   - Interpretation: Prone to overfitting, performs well on training but may not generalize

2. Ridge Regression:
   - Bias: Slightly higher than Linear Regression (due to coefficient shrinkage)
   - Variance: Lower than Linear Regression (regularization reduces sensitivity to data)
   - Interpretation: Better generalization, trades increased bias for reduced variance

3. Lasso Regression:
   - Bias: Can be higher due to feature selection (some coefficients forced to zero)
   - Variance: Lower than Ridge in high-dimensional settings
   - Interpretation: Performs feature selection, useful when many irrelevant features

4. Elastic Net:
   - Bias: Moderate (combines properties of Ridge and Lasso)
   - Variance: Balanced approach
   - Interpretation: Compromise between Ridge and Lasso, handles correlated features well
""")


BIAS-VARIANCE ANALYSIS

Bias-Variance Trade-off Analysis:

1. Linear Regression:
   - Bias: Low (flexible model that fits training data well)
   - Variance: High (small changes in data can cause large changes in predictions)
   - Interpretation: Prone to overfitting, performs well on training but may not generalize

2. Ridge Regression:
   - Bias: Slightly higher than Linear Regression (due to coefficient shrinkage)
   - Variance: Lower than Linear Regression (regularization reduces sensitivity to data)
   - Interpretation: Better generalization, trades increased bias for reduced variance

3. Lasso Regression:
   - Bias: Can be higher due to feature selection (some coefficients forced to zero)
   - Variance: Lower than Ridge in high-dimensional settings
   - Interpretation: Performs feature selection, useful when many irrelevant features

4. Elastic Net:
   - Bias: Moderate (combines properties of Ridge and Lasso)
   - Variance: Balanced approach
   - Interpretation: Compromise betwee

In [16]:
# ============================================================================
# 14. CONCLUSION AND FINAL OBSERVATIONS
# ============================================================================

print("\n" + "="*80)
print("CONCLUSION AND FINAL OBSERVATIONS")
print("="*80)

# Find the best model based on test R²
best_model_idx = np.argmax([r2_score(y_test, pred) for pred in test_predictions.values()])
best_model_name = list(test_predictions.keys())[best_model_idx]
best_model_r2 = r2_score(y_test, test_predictions[best_model_name])

print(f"""
Based on the comprehensive analysis of all regression models:

Best Performing Model: {best_model_name}
Best Test R² Score: {best_model_r2:.4f}

Key Findings:
-------------
1. Model Performance Summary:
   - Linear Regression: R² = {r2_score(y_test, test_predictions['Linear Regression']):.4f}
   - Ridge Regression: R² = {r2_score(y_test, test_predictions['Ridge Regression']):.4f}
   - Lasso Regression: R² = {r2_score(y_test, test_predictions['Lasso Regression']):.4f}
   - Elastic Net: R² = {r2_score(y_test, test_predictions['Elastic Net Regression']):.4f}

2. Regularization Impact:
   - Ridge regression successfully reduced overfitting by shrinking coefficients
   - Lasso performed feature selection, identifying the most important predictors
   - Elastic Net provided a balanced approach between Ridge and Lasso

3. Optimal Hyperparameters:
   - Ridge (alpha={cv_results['Ridge']['best_params']['alpha']}): Optimal regularization strength
   - Lasso (alpha={cv_results['Lasso']['best_params']['alpha']}): Best for feature selection
   - Elastic Net (alpha={cv_results['Elastic Net']['best_params']['alpha']},
                  l1_ratio={cv_results['Elastic Net']['best_params']['l1_ratio']}): Optimal mix

4. Trade-offs:
   - Accuracy vs. Complexity: Regularized models show better generalization
   - Interpretability vs. Performance: Lasso provides sparse, interpretable models
   - Training Time vs. Accuracy: Grid search increases training time but improves results

Recommendation:
--------------
{best_model_name} is recommended as the final model based on its superior test performance
and good generalization characteristics. The chosen hyperparameters provide the optimal
balance between bias and variance for this dataset.
""")

# Save all results to a CSV file for reference
results_summary = pd.concat([
    table1_df.assign(Table='Table 1: Hyperparameter Tuning'),
    table2_df.assign(Table='Table 2: CV Performance'),
    table3_df.assign(Table='Table 3: Test Performance'),
    table4_df.assign(Table='Table 4: Coefficients')
], ignore_index=True)

results_summary.to_csv('regression_analysis_results.csv', index=False)
print("\nAll results have been saved to 'regression_analysis_results.csv'")
print("="*80)


CONCLUSION AND FINAL OBSERVATIONS

Based on the comprehensive analysis of all regression models:

Best Performing Model: Lasso Regression
Best Test R² Score: -16484921.0068

Key Findings:
-------------
1. Model Performance Summary:
   - Linear Regression: R² = -16577344.4450
   - Ridge Regression: R² = -16537153.0455
   - Lasso Regression: R² = -16484921.0068
   - Elastic Net: R² = -16505769.1748

2. Regularization Impact:
   - Ridge regression successfully reduced overfitting by shrinking coefficients
   - Lasso performed feature selection, identifying the most important predictors
   - Elastic Net provided a balanced approach between Ridge and Lasso

3. Optimal Hyperparameters:
   - Ridge (alpha=1): Optimal regularization strength
   - Lasso (alpha=0.1): Best for feature selection
   - Elastic Net (alpha=0.01, 
                  l1_ratio=0.8): Optimal mix

4. Trade-offs:
   - Accuracy vs. Complexity: Regularized models show better generalization
   - Interpretability vs. Performance